# CHEM 60 - March 20th, 2024 (Applying ODEs to Ozone)

Okay. What was that all for? Presumably, you're here because you wrote a function that solves for a system of ODEs to simulate chemistry and... you want to know why we care so much about doing it as chemists. We weren't even using a real chemical mechanism before.

The paper that Yuki and Max picked for us in the first class is a great application of numerically solving a system of ODEs to answer some cool and important questions about chemistry.

If you are excited about this content, I encourage you to give it a read. [Chlorine activation and enhanced ozone depletion induced by wildfire aerosol](https://www.nature.com/articles/s41586-022-05683-0). It is not a required read this week though, because, to appreciate all that they did will require a fair bit of background knowledge on how we simulate chemistry in the atmosphere (hey, that's what I do!). This notebook will talk through how we think about this for gas-phase reactions. The linked Nature paper extends this to aerosol chemistry which adds a bunch of complexity that I am leaving out here (feel free to ask me later about this though). The seminar example I linked in the in-class notebook is also looking at an aerosol/droplet application of very related chemistry (that happens to be more important at the surface than the stratospheric chemistry we'll talk through here). This notebook should be a much more beginner friendly alternative to reading the paper.

#Imports

Here are the Python imports for this one. Run the below code block to get started.

In [ ]:
# Standard library imports
from IPython.display import YouTubeVideo
import math as m

# Third party imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp


# This part of the code block is telling matplotlib to make certain font sizes exra, extra large by default
# Here is where I list what parametres I want to set new defaults for
params = {'legend.fontsize': 'xx-large',
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large'}
# This line updates the default parameters of pyplot (to use our larger fonts)
plt.rcParams.update(params)

# How do we model the (changing) chemical composition of the atmosphere?
First, how do we even think about it?

![How do we think about how the atmosphere changes? http://kavassalis.space/s/SlideMa.png](https://static1.squarespace.com/static/61ae9885d7103a326c42a062/t/6206f16b7252ce26eadc8f28/1644622187707/SlideMa.png)

If we want to think about how some compound, $X$, changes in the atmosphere, we typically use what's known as a box-model approach (sometimes called a mass-balance approach). The basic idea is as follows:

1. Pick the region of the atmosphere we're interested in that is relevant for our compound of interest. Is our volume an entire city? A few metres? The whole Earth? This is non-trivial, as I'll mention below.

2. Assume the volume is well-mixed (no concentration gradients). This means you have to pick a box small enough that this is true for the compounds you're interested in.

3. Assume any relevant processes happen uniformly within the box.

Then, define the sources (inputs/inflows) into the box and the sinks (outputs/outflows). Examples of sources can be direct emissions (ex. cars emit $NO$), advection (ex. wind blows sea salt to shore), or chemical reactions within the box itself (ex. $O_3$ is formed chemically, not emitted).

Sinks can be things like deposition (ex. particles or molecules are 'dry deposited' when they settle on a surface and don't re-enter the air - 'wet deposition' is rain, so things can also be rained out), chemical reactions (ex. a pollutant like benzene can be removed by chemical transformation through reaction with an oxidant), advection again (things can blow away).

We can do our best to represent all of these processes in mechanistic models where we essentially solve this equation:
\begin{align}
\frac{d[X]}{dt}=
+ Emission+Advection+Chemistry+Deposition
\end{align}

$Emissions$ and $Deposition$ are included in models through empirical parameterizations (equations built from data and a bit of mechanistic insight).  The spatial scale of the model determines how $Advection$ is handled - models with "physics engines" or "dynamic cores" can solve for advection deterministically (wind is simulated with PDEs) but in some models it's included as an empirical parameterization. $Chemistry$ is included by writing the relevant reactions as a system of ordinary differential equations and solving them numerically. I'm leaving out for now what is often called "mixing" - essentially how models handle turbulent mixing, which is very scale dependent (and always some form of parameterization). The treatment of turbulence in models helps inform how vertical transport works (we don't need this right now).

This box model approach scales to pretty much all scales (including the entire Earth), because even though we can't assume large areas are well simulated by a single box, we *can* assume several boxes all stacked on-top and next to each other can represent that kind of space. All we need to do then is describe the exchange between boxes (typically advection or "mixing" based).

![0D vs 1D vs 3D models (it's all boxes): http://kavassalis.space/s/SlideMd.png](https://static1.squarespace.com/static/61ae9885d7103a326c42a062/t/6208923f6d10ea045a8986bb/1644728895510/SlideMd.png)

So, understanding the basic way a 0-D box model works is important because not only does it serve as our heuristic for thinking about how atmospheric change occurs, but because it's also the fundamental building block of sophisticated chemical transport models (a model that solves a big system of coupled ODEs representing atmospheric chemistry coupled with things to appropriately move the air around <- I am being vague with "things" (I teach other courses on this though!)).


# How do we model chemistry for a big system like the atmosphere?

1. Define the chemical reactions relevant for regulating the compounds in the atmosphere of interest
2. Identify what other relevant processes impact all the chemical compounds in those reactions
3. Include all of those things within the appropriate differential equations (they're not always ordinary...)

![How do we think about how the atmosphere changes? Sources vs Sinks: http://kavassalis.space/s/SlideMc.png](https://static1.squarespace.com/static/61ae9885d7103a326c42a062/t/62088b1182cb265dbd07f6af/1644727058139/SlideMc.png)

Going back to the simple diagram, the non-chemical sources/sinks we need to think about for most atmospheric chemistry problems are:

1. Advection. This can be a source or sink as wind can blow pollutants into or out of the region we're interested in. Understanding how this impacts air quality depends in a big way on the chemical lifetime of the compounds we're trying to understand. If we are focused on very short-lived chemical compounds (things that will really quickly react or deposit on a surface), we may not care much about advection. If we are focused on longer-lived compounds, then horizontal transport could become really important. Air quality chemistry involves extremely short-lived and  long-lived compounds, so advection is certainly important, but there are situations where the pollutants we're interested in are entirely locally produced and wind plays a minor role in regulating their concentrations.

2. Deposition is an important sink for many key pollutants. Ozone is readily taken up by plants, for example, and ozone concentrations are strongly impacted by the ability for vegetation to remove it from the air. In the stratosphere, deposition onto particles is very important! All surfaces have some ability to 'dry deposit' pollutants.

3. Emissions are essential for primary pollutants - these are compounds emitted directly from a source, like many volatile organic compounds, NO, carbon monoxide, some types of aerosols. Secondary pollutants are only formed through chemical reactions (things like ozone, secondary organic aerosols) and have no emissions (but are obviously still dependent on them).

4. Vertical transport. This is distinct from advection (which is horizontal transport, where the rate of removal - or addition - of a pollutant is a function of horizontal wind speed). Vertical transport can occur through large convective updrafts (think the feeling in the air that happens when a storm is forming) or through turbulent transport. These processes are mathematically represented in very different ways (and vertical transport is much more problematic to get right in models than advection) but are essential for many kinds of simulations.

# Stratospheric Ozone

Since stratospheric chemistry is sometimes in the Mudd core, this will hopefully be somewhat familar (but I am going to go over it again). Before we do the chemistry (in case I somehow didn't manage to play this for you in class), this delightful musical number from the UN's Environmental Programme on stratospheric ozone is well worth watching to remind you what stratospheric ozone is all about.

In [ ]:
YouTubeVideo('3xJthmuapBg', width=800, height=600)

Anyways... let's get into the chemistry that actually makes ozone.

# The Chapman Mechanism

![How does chemistry control composition: http://kavassalis.space/s/Slide5.png](https://static1.squarespace.com/static/61ae9885d7103a326c42a062/t/6205e3a24735485171e29c42/1644553122905/Slide5.png)

If you've studied the ozone before, this is the chemistry you have seen - these chemical reactions create stratospheric ozone (the ozone layer). This mechanism (a mechanism is essentially a list of chemical reactions) is extremely important for many reasons, perhaps most notably because it allowed for life to evolve on land! This is because ozone absorbs ultraviolet radiation that would otherwise make it to the surface (and DNA doesn't fair well when it interacts with too much light <320 nm).

\begin{align}
O_{2} + h\nu & \rightarrow O + O(^{1}D) & (\lambda<240nm) &&(R1) \\
O + O_{2} + M & \rightarrow O_{3} + M &  &&(R2) \\
O_{3} + h\nu&  \rightarrow O_{2} + O(^{1}D) & (\lambda<320nm) &&(R3) \\
O_{3} + O& \rightarrow 2O_{2} &  &&(R4)
\end{align}

The rate that these reactions proceed at is important, and we have a nice way of writing that *rate* down as a function of some rate constant (a measure of how likely a reaction is to happen) and the concentration of reactants.

\begin{align}
\text{Rate 1} & = J_1[O_2] \\
\text{Rate 2} & = k_2[O][O_2][M] \\
\text{Rate 3} & = J_3[O_3] \\
\text{Rate 4} & = k_4[O_3][O] \\
\end{align}


The rate constants are $J_1$ for $R1$, $k_2$ for $R2$, $J_3$ for $R3$, and $k_4$ for $R4$. $J_{1}$ and $J_{3}$ have units of $s^{-1}$ (photon density is incorporated into rate constants for reactions with light), $k_{2}$ in units of $cm^{6}molecules^{-2}s^{-1}$, and $k_{4}$ in units of $cm^{3}molecules^{-1}s^{-1}$.

Notation note: Rate constants for photolysis reactions (reactions of molecule + light) are often labeled with $J$s and rate constants for molecular reactions (reactions of molecule + molecule + molecule) are often labeled with $k$s.


The photolysis rate constants (J-values) are dependent on the amount of light within specific wavelength bands arriving at different heights in the atmosphere. This light comes from the sun and is scattered and absorbed by molecules in the atmosphere, so J-values are typically decreasing as you approach the surface of the Earth. These aren't measured by satellites, but they can be modelled. The values we will be using are output from the <a href="http://cprm.acom.ucar.edu/Models/TUV/Interactive_TUV/">TUV calculator</a>, a free, online tool for modelling photolysis frequencies and actinic flux (it's very commonly used in atmospheric chemistry research). I saved output already that we can load in as .txt files but I wanted you to know where it came from.

In [ ]:
Chapman_df = pd.DataFrame()
# download a .txt file containing photolysis rate constants for O2 -> O + O from 0 - 50km~
Chapman_df['J1'] = np.loadtxt(fname = "http://kavassalis.space/s/J1.txt")
# download a .txt file containing photolysis rate constants for O3 -> O2 + O(1D) from 0 - 50km~
Chapman_df['J3'] = np.loadtxt(fname = "http://kavassalis.space/s/J3.txt")

Molecular rate constants are generally functions of temperature and pressure and have expressions that can be looked up. <a href="https://kinetics.nist.gov/kinetics/">NIST</a> is the standard source for these. Below, I've pulled (perhaps overly simplified) expressions for $k_2$ and $k_4$ in the Chapman equations.

$k_{2} = 6\times10^{-34}(T/300)^{-2.3}$

$k_{4} = 8\times10^{-12}exp(-2060/T)$

The rate expression above is a function of temperature, so we should grab the temperature profile data. Let's start by looking at a real temperature profile of the atmosphere. We'll load in an observed vertical profile of temperature vs pressure over the Claremont area from the <a href="https://airs.jpl.nasa.gov/data/products/v7-L2-L3/">Atmospheric Infrared Sounder</a>, or AIRS, instrument.

To quote Joao Teixeira, JPL—AIRS Science Team Leader, “AIRS revolutionized weather prediction by providing—for the first time—a three-dimensional picture of the atmosphere”. AIRS has long been a critical instrument on NASA's Earth Observing System (EOS) polar-orbiting platform and there is some nice history of EOS recorded <a href="https://drive.google.com/file/d/1RqLKmo_41WdhPxFAncCmuRBlZWwkOB3c/view?usp=sharing">here</a> as well as collection of data visualizations from its <a href="https://airs.jpl.nasa.gov/news/189/20-years-of-airs-a-story-in-visualizations/">20 year history</a>. It's hard to overstate the importance of this dataset to atmospheric science (we're just using it for some very simple data products, but it does a lot for us).

In [ ]:
AIRS_df = pd.read_csv('https://kavassalis.space/s/g4averagedAIRS3STD_7_0_Temperature_A20220208-20220208118W_33N_117W_34N.csv',header=8)
# and now I remember it didn't come with altitude
R = 8.31; Ma = 29; g = 9.8
Chapman_df['Altitude'] = [-m.log(AIRS_df['StdPressureLev'][z]/AIRS_df['StdPressureLev'][0])*(R*AIRS_df['AIRS3STD_7_0_Temperature_A'][z])/(Ma*g) for z in range(len(AIRS_df['StdPressureLev']))]

Then we can use that temperature data to make rate constants for the two molecular reactions.

In [ ]:
Chapman_df['k2'] = 6E-34*(AIRS_df['AIRS3STD_7_0_Temperature_A']/300)**-2.3
Chapman_df['k4'] = 8E-12*m.exp(1)**(-2060/AIRS_df['AIRS3STD_7_0_Temperature_A'])

Now let's plot all four rate constants.

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(8,10))
plt.subplots_adjust(hspace = 0.35)

axs[0, 0].set_ylabel('Altitude (km)')
axs[0, 0].set_xlabel('$J_1$ \n ($s^{-1}$)')
axs[0, 0].plot(Chapman_df['J1'],Chapman_df['Altitude'], 'k', linewidth=3.0)

axs[1, 0].set_ylabel('Altitude (km)',)
axs[1, 0].set_xlabel('$J_3$ \n ($s^{-1}$)')
axs[1, 0].plot(Chapman_df['J1'],Chapman_df['Altitude'], 'k', linewidth=3.0)

axs[0, 1].set_xlabel('$k_2$ \n ($cm^{6 }\hspace{.5} molecules^{-2}\hspace{.5} s^{-1}$)')
axs[0, 1].plot(Chapman_df['k2'],Chapman_df['Altitude'], 'k', linewidth=3.0)

# axs[1, 1].set_ylabel('Altitude (km)')
axs[1, 1].set_xlabel('$k_4$ \n ($cm^{3}\hspace{.5} molecules^{-1}\hspace{.5} s^{-1}$)')
axs[1, 1].plot(Chapman_df['k4'],Chapman_df['Altitude'], 'k', linewidth=3.0)

plt.show()

Before we discuss why these profiles are different (and that those differences are important), let's plot $O_2$ versus altitude too...

In [ ]:
Av = 6.023e23 # molecules mol-1 is Avogadro's number
Chapman_df['Na'] = Av*100*AIRS_df['StdPressureLev']/(R*AIRS_df['AIRS3STD_7_0_Temperature_A'])/(100**3)
Chapman_df['O2'] = 0.21*Chapman_df['Na'] # this one is easy, because oxygen is ~21% of the atmosphere

And a plot

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12,8))

axs[0].set_ylabel('Altitude (km)')
axs[0].set_xlabel('$J_1$ ($s^{-1}$)')
axs[0].plot(Chapman_df['J1'],Chapman_df['Altitude'], 'k', linewidth=7)

axs[1].plot(Chapman_df['O2'], Chapman_df['Altitude'], c='#1b9e77', linestyle='-', linewidth=7, marker='o', markersize=15)
axs[1].set_ylabel('Altitude (km)')
axs[1].set_xlabel('[$O_2$] (molecules/$cm^3$)')

plt.show()

Now looking at the rate constants and the $O_2$ picture, we can get a real sense of why the ozone layer forms where it does. While the photochemical rate constants are highest at high altitudes, molecular oxygen is really sparse up there. The ozone layer forms in this sweet-spot where there is both enough oxygen and high energy photons available for reactions to occur. What do the profiles of $k_2$ and $k_4$ contribute here? (look at what is a source of ozone and what is a sink).

# If we know all this, can we simulate (model) ozone?

We can write down the differential equations that govern this system as,

\begin{align}
\frac{d[O_3]}{dt} &= k_2[O][O_2][M] - J_3[O_3] - k_4[O_3][O]\\
\frac{d[O]}{dt} &= 2 J_1[O_2] - k_2[O][O_2][M]- k_4[O_3][O]\\
\frac{d[O_2]}{dt} &= -J_1[O_2] - k_2[O][O_2][M] + k_4[O_3][O]\\
\end{align}

And then solve the system of equations!

We are going to make things much easier for us by using a pre-built integrator in this notebook. First, let's write a function that just expresses the above differential equations.

In [ ]:
def dO3dt_Chapman(t, J1, k2, J3, k4, M, Chem):
    O3, O, O2 = Chem

    dO3dt =        + k2*O*O2*M - J3*O3 -  k4*O3*O
    dOdt = 2*J1*O2 - k2*O*O2*M + J3*O3 -  k4*O3*O
    dO2dt = 0 # we're not really changing the amount of O2 with this chemistry
    # (think about the relative amounts of O2 vs the other components and the reaction rates)

    return dO3dt, dOdt, dO2dt

Define parameters. Let's test it for a given height (z = 10).

In [ ]:
z = 10
# select rate constants for each height, z
J1 = Chapman_df['J1'][z]
k2 = Chapman_df['k2'][z]
J3 = Chapman_df['J3'][z]
k4 = Chapman_df['k4'][z]

#This is the "M" term - any third body (also known as the number density)
M = Chapman_df['Na'][z]

# Initial conditions
Chem0 = 0, 0, Chapman_df['O2'][z]

# Time span
t0 = 0
tf = 1E12 # this is time in seconds!

# Call the solve_ivp function
result = solve_ivp(lambda t, y: dO3dt_Chapman(t, J1, k2, J3, k4, M, y), (t0, tf), Chem0, method='BDF')

The `result` object is a *dictionary* which contains various information about the solution. result.y will be a 2D array where we have our simulated chemicals stored. We can pull it apart like so:

In [ ]:
O3_res, O_res, O2_res = result.y

And now a plot:

In [ ]:
plt.plot(result.t, O3_res, c='#7570b3', lw=5)
plt.xlabel('Time (s)')
plt.ylabel('$[O_3]$ (molecules/$cm^3$)')
plt.title('[$O_3$] at '+str(round(Chapman_df['Altitude'][z],2))+" km")
plt.show()

Some things to note, the system reaches a steady-state eventually but it's a bit artificial (we are assuming that it is always day-time!).

Also, the pre-built integrator we used was fast!

Now, if we want to understand the paper Max and Yuki picked, we don't just want to see the concentration at one height, we want it for all the heights.

In [ ]:
o3_final_concentration = []

for z in range(len(Chapman_df)):
    # Define height-dependent parameters here
    J1 = Chapman_df['J1'][z]
    k2 = Chapman_df['k2'][z]
    J3 = Chapman_df['J3'][z]
    k4 = Chapman_df['k4'][z]

    M = Chapman_df['Na'][z]

    # Initial conditions
    Chem0 = 0, 0, Chapman_df['O2'][z]

    result = solve_ivp(lambda t, y: dO3dt_Chapman(t, J1, k2, J3, k4,M, y), (t0, tf), Chem0, method='BDF')
    O3_res, _, _ = result.y
    o3_final = O3_res[-1]  # Get final concentration
    o3_final_concentration.append(o3_final)

Now plot the final concentration against height

In [ ]:
plt.figure(figsize=(6,8))
plt.plot(o3_final_concentration, Chapman_df['Altitude'], c='#7570b3', lw=5)
plt.xlabel('[$O_3$] (molecules/$cm^3$)')
plt.ylabel('Altitude (km)')
plt.show()

# Comparing simulation to observation

The paper includes a lot more chemistry than we did (and also some hidden physics that we can talk about for anyone curious), but the basic idea is the same. They simulate the chemistry by solving a system of ODEs and then compare their simulations to observations to say if they included the right chemistry.

We can do that too - I pre-downloaded the ozone profile the AIRS instrument took at the same time as the temperature one to see how close our very simple model gets us.

In [ ]:
O3_obs = pd.read_csv('http://kavassalis.space/s/g4averagedAIRS3STD_7_0_O3_VMR_A20220208-20220208118W_33N_117W_34N.csv',header=8)
Chapman_df['O3_observation'] = O3_obs['AIRS3STD_7_0_O3_VMR_A']*Chapman_df['Na']/10**9 # unit converstion to make them the same

And plotting the observations vs simulation...

In [ ]:
fig, ax = plt.subplots(1, figsize=(6,8))
ax.plot(o3_final_concentration, Chapman_df['Altitude'], c='#7570b3', lw=5, label = "simulation")
ax.plot(Chapman_df['O3_observation'], Chapman_df['Altitude'], 'k', marker = 'o', markersize=10, lw=0, label="observation")
ax.set_ylabel('Altitude (km)')
ax.set_xlabel('[$O_3$] (molecules/$cm^3$)')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.show()

Uh oh. This isn't close at all! There are several reasons why our Chapman-only model would over-predict stratospheric ozone. The first one being that the Chapman mechanism doesn't contain all of the relevant reactions that occur in the stratosphere. The natural <b>catalytic destruction</b> of ozone by hydrogen oxide radicals ($HO_x$) and nitrogen oxides ($NO_x$) has been neglected. We have also not included catalytic destruction due to chlorine and bromine radicals. Ozone gets made through different chemistry near the surface (and we've ignored that). Transport is also important in the lower levels of our model, so pretending the layers of the model are only dictated by local chemistry and no vertical mixing isn't especially realistic. Plus, the surface chemistry that the linked paper above deals with is entirely missing here. The gas-phase reactions that are missing are easy to add and will improve this simulation a lot. By only including a portion of the chemical production and loss terms (and none of the other terms talked about at the top of the notebook), we aren't going to make a great model.

Okay, so, our model might be "wrong" if we don't include all the relevant processes, *but* this very simplified picture was *enough* to predict the *existence* of an ozone layer and get the height it should form roughly correct in the first place. That's practically worth part of a <a href="https://www.nobelprize.org/prizes/chemistry/1995/summary/">Nobel prize</a>!

While a lot of our understanding of this chemistry comes from the 1970s-1990s, there are still aspects of the chemical mechanism we are refinining. This is how cool papers still get published on this topic. The basic thing you need is: know the elementary steps of the reactions you think are happening, have some rate constants to try, and solve a big system of ODEs then compare it with observations to see if we did better than what came before. That's the heart of how this research works.



---



You don't need to submit this one! It's just to help you on the homework. If you want to plot this version of the simulation in your homework to test to make sure your Runge Kutta function is working, don't copy and paste everything from here. Just grab the object(s) you want.

In [ ]:
o3_final_concentration

ie. o3_final_concentration = [0.0023858928736674263,
 0.06071363662509872,
 1.582219324712108,
 1627.9340937564225,
 139648.75344120522,
 11517210.028486125,
 913901179.8933201,
 44511539032.95028,
 284344217708.0456,
 913720564411.8662,
 3096846233438.1147,
 12788469781387.598,
 24174145549362.96,
 34377116198243.97,
 39629290775354.8,
 37865413235680.25,
 31016020448599.6,
 21552473712677.062,
 12276706280990.014,
 6284668650626.595,
 1629543704045.0498,
 575491476822.037,
 304702971826.67316,
 148021771654.3029]